In [1]:
#打包答案时候检查是否漏音频文件
import os
import re

def check_missing_files(folder_path):
    # 正则表达式匹配文件名：synthesized_speech_数字.wav
    pattern = re.compile(r'synthesized_speech_(\d+)\.wav')
    
    # 存储找到的数字
    found_numbers = []

    # 遍历文件夹中的所有文件
    for filename in os.listdir(folder_path):
        match = pattern.match(filename)
        if match:
            num = int(match.group(1))
            if 1 <= num <= 200:
                found_numbers.append(num)

    # 去重并排序
    found_numbers = sorted(set(found_numbers))

    # 所有应该存在的编号
    expected_numbers = set(range(1, 201))

    # 找出缺失的编号
    missing_numbers = sorted(expected_numbers - set(found_numbers))

    # 输出结果
    if missing_numbers:
        print(f"缺失的文件编号（{len(missing_numbers)} 个）：")
        print(missing_numbers)
    else:
        print("所有文件都存在，没有缺失。")

    print(f"\n共找到 {len(found_numbers)} 个有效文件。")

# 使用示例
if __name__ == "__main__":
    folder = "./aha!-result"
    
    if os.path.isdir(folder):
        check_missing_files(folder)
    else:
        print("错误：不是有效的文件夹路径！")

缺失的文件编号（200 个）：
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200]

共找到 0 个有效文件。


In [2]:
#检查不止是中文的文本，量太少了手动改了
import pandas as pd
import re

# 读取CSV文件
file_path = './AISumerCamp_audio_generation_fight/aigc_speech_generation_tasks/aigc_speech_generation_tasks.csv'  # 替换为你的CSV文件路径
df = pd.read_csv(file_path)

# 指定要检查的列名
column_name = 'text'  # 替换为你要检查的列名

# 检查列是否存在
if column_name not in df.columns:
    raise ValueError(f"列名 '{column_name}' 不存在于CSV文件中。")

# 定义一个函数：判断字符串是否包含英文字母（a-z, A-Z）
def contains_english(text):
    if pd.isna(text):  # 处理空值
        return False
    return bool(re.search(r'[a-zA-Z]', str(text)))

def contains_non_chinese(text):
    """
    判断字符串是否包含非中文内容（如英文字母、数字、日文、韩文等）
    原理：如果整个字符串不能被“中文+常见符号”完全匹配，则认为包含非中文
    """
    if pd.isna(text):
        return False
    
    text = str(text).strip()
    if not text:
        return False

    # 匹配：中文字符 + 中文/英文标点 + 空白字符
    # 如果整个字符串都匹配这个模式 → 是纯中文
    # 否则 → 包含非中文内容
    pure_chinese_pattern = r'^[\u4e00-\u9fff\u3400-\u4dbf\uf900-\ufaff\p{P}\s0-9]+$'

    # 注意：Python 的 re 不支持 \p{P}，改用显式标点
    pure_chinese_pattern = r'^[\u4e00-\u9fff\u3400-\u4dbf\uf900-\ufaff\。\，\！\？\；\：\“\”\‘\’\（\）\【\】\《\》\、\s\.,!?;:"\'()\[\]{}<>\\|`~@#$%^&*\+=_\-0-9]+$'

    # 使用 re.match 检查是否“完全匹配”纯中文模式
    is_pure = bool(re.fullmatch(pure_chinese_pattern, text))

    return not is_pure  # 不是纯中文 → 包含非中文内容

# 找出该列中包含英文字符的行
# filtered_df = df[df[column_name].apply(contains_english)]
filtered_df = df[df[column_name].apply(contains_non_chinese)]

with pd.option_context('display.max_colwidth', None):
    print(filtered_df[column_name])
    
#包含英文的项
#需要纠正的 8、60、80、116、172（9、61、81、117、173）
# 8                                   每股美国存托股（ADS）基本和摊薄亏损均为人民币0.75元（约合0.11美元），而上年同期每股ADS基本和摊薄亏损均为人民币0.88元。
# 11                                                                                 预定网约车，车型选SUV，司机需有儿童座椅
# 19                                                                                     租用共享汽车，需要自动挡SUV车型
# 29                                                                      给我购买一台新款苹果电脑，配上16GB内存和512GB硬盘的那种
# 39                                                     随着人工智能技术的不断进步，越来越多的企业开始将AI应用于医疗诊断、金融分析和制造业自动化等领域。
# 51                                      在这种情况下，首先要确保你使用了合适的优化算法，例如Adam或RMSprop。此外，调整学习率和批次大小也会对结果产生显著影响。
# 60                                                                              订3月15日MU5115上海飞北京经济舱靠窗座位
# 80                                                                          购买一台电视，65英寸，4k分辨率，直接从我的银行卡扣款
# 105    G女士收到一条短信写着快递丢失点进链接可以帮助理赔退款，G女士加了客服的联系方式，并指导她下载某会议APP，视频指导她先发送支付宝口令红包，并告知马上即可给她退款，G女士打完款后被拉黑遂意识被骗
# 116          成年人的肺活量男性平均为 3500-4000 mL，女性为 2500-3000 mL，吸烟或患有慢性呼吸道疾病的人群通常会低于此标准。建议定期进行体检，并在有任何异常时及时咨询医生。
# 172                          甲状旁腺激素受体（PTH1R）广泛分布于骨和肾脏组织中，当PTH与其结合后，可激活腺苷酸环化酶系统，引发一系列信号转导反应，调节钙磷平衡及骨代谢活动。

# 9                                   每股美国存托股（A D S）基本和摊薄亏损均为人民币0.75元（约合0.11美元），而上年同期每股'A D S'基本和摊薄亏损均为人民币0.88元。
# 51                                      在这种情况下，首先要确保你使用了合适的优化算法，例如Adam或R M S prop。此外，调整学习率和批次大小也会对结果产生显著影响。
# 61                                                                              订3月15日'M U 5 1 1 5'上海飞北京经济舱靠窗座位
# 81                                                                          购买一台电视，65英寸，4 K分辨率，直接从我的银行卡扣款
# 117          成年人的肺活量男性平均为 3500-4000 毫升，女性为 2500-3000 毫升，吸烟或患有慢性呼吸道疾病的人群通常会低于此标准。建议定期进行体检，并在有任何异常时及时咨询医生。
# 173                          甲状旁腺激素受体（P T H 1 R）广泛分布于骨和肾脏组织中，当'P T H'与其结合后，可激活腺苷酸环化酶系统，引发一系列信号转导反应，调节钙磷平衡及骨代谢活动。


8                                                                                                                                                                                                                                                                   每股美国存托股（ADS）基本和摊薄亏损均为人民币0.75元（约合0.11美元），而上年同期每股ADS基本和摊薄亏损均为人民币0.88元。
11                                                                                                                                                                                                                                                                                                                 预定网约车，车型选SUV，司机需有儿童座椅
19                                                                                                                                                                                                                                                                                                                     租用共享汽车，需要自动挡SUV车型
29           

In [5]:
#统计一下音长数据
import os
import pandas as pd
import librosa

# -------------------------------
# 用户配置
# -------------------------------
fold_root = "./AISumerCamp_audio_generation_fight/aigc_speech_generation_tasks/"               # result.csv 所在目录
audio_dir = "./pre_deal/fix/"    # 音频文件所在目录（可改为增强后目录）
csv_file = os.path.join(fold_root, "aigc_speech_generation_tasks.csv")

# 创建 0～10 秒的统计桶（bins）
bins = {f"{i}-{i+1}s": 0 for i in range(10)}  # {'0-1s':0, '1-2s':0, ..., '9-10s':0}

# -------------------------------
# 主逻辑：读取 CSV → 获取音频 → 统计时长分布
# -------------------------------
if __name__ == "__main__":
    # 1. 读取 CSV
    if not os.path.exists(csv_file):
        raise FileNotFoundError(f"CSV file not found: {csv_file}")

    task = pd.read_csv(csv_file)

    if 'reference_speech' not in task.columns:
        raise ValueError("CSV 文件中缺少 'reference_speech' 列！")

    # 2. 遍历每一行
    count_total = 0
    count_invalid = 0

    for idx, row in task.iterrows():
        ref_name = row['reference_speech']

        # 处理扩展名
        base_name, ext = os.path.splitext(ref_name)
        if ext == "":
            audio_file = base_name + ".wav"
        else:
            audio_file = ref_name

        audio_path = os.path.join(audio_dir, audio_file)

        if not os.path.exists(audio_path):
            print(f"⚠️  File not found: {audio_path}")
            count_invalid += 1
            continue

        # 读取音频时长
        try:
            duration = librosa.get_duration(filename=audio_path)
            count_total += 1
        except Exception as e:
            print(f"❌ Error reading {audio_path}: {e}")
            count_invalid += 1
            continue

        # 判断落在哪个区间：0-1s, 1-2s, ..., 9-10s
        if 0 <= duration < 10:
            second = int(duration)  # 向下取整，比如 3.7s → 第3秒区间
            key = f"{second}-{second+1}s"
            bins[key] += 1
        elif duration >= 10:
            print(f"💡 {audio_file}: {duration:.2f}s (>10s), skipped in histogram")
        else:
            print(f"⚠️  {audio_file}: invalid duration {duration}")

    # 3. 打印结果
    print("\n" + "="*40)
    print("Duration Distribution (0-10s)")
    print("="*40)
    for interval, count in bins.items():
        print(f"{interval:6s}: {count}")

    print("-" * 40)
    print(f"Total valid audios: {count_total}")
    if count_invalid > 0:
        print(f"Invalid/missing: {count_invalid}")
    print(f"Audio directory: {audio_dir}")
    print("="*40)

/tmp/ipykernel_7858/1377538206.py:52: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=audio_path)


💡 reference_13.wav: 30.00s (>10s), skipped in histogram
💡 reference_17.wav: 30.00s (>10s), skipped in histogram
💡 reference_26.wav: 30.00s (>10s), skipped in histogram
💡 reference_34.wav: 10.88s (>10s), skipped in histogram
💡 reference_42.wav: 15.09s (>10s), skipped in histogram
💡 reference_44.wav: 17.56s (>10s), skipped in histogram
💡 reference_57.wav: 30.00s (>10s), skipped in histogram
💡 reference_58.wav: 13.50s (>10s), skipped in histogram
💡 reference_70.wav: 30.00s (>10s), skipped in histogram
💡 reference_77.wav: 10.20s (>10s), skipped in histogram
💡 reference_79.wav: 30.00s (>10s), skipped in histogram
💡 reference_82.wav: 30.00s (>10s), skipped in histogram
💡 reference_86.wav: 30.00s (>10s), skipped in histogram
💡 reference_98.wav: 27.77s (>10s), skipped in histogram
💡 reference_104.wav: 30.00s (>10s), skipped in histogram
💡 reference_105.wav: 30.00s (>10s), skipped in histogram
💡 reference_115.wav: 15.09s (>10s), skipped in histogram
💡 reference_117.wav: 30.00s (>10s), skipped i

In [ ]:
!pip install torch torchaudio
!pip install pydub

In [7]:
#查询人声占多少毫秒

import os
import pandas as pd
import torch
import torchaudio
import numpy as np

# -------------------------------
# 用户配置
# -------------------------------
fold_root = "./AISumerCamp_audio_generation_fight/aigc_speech_generation_tasks/"              # result.csv 所在目录
# audio_dir = "./result/"    # 音频文件所在目录（可改为增强后目录）
audio_dir = "./pre_deal/fix/"    # 增强后的音频文件所在目录（可改为增强后目录）

csv_file = os.path.join(fold_root, "aigc_speech_generation_tasks.csv")

# 加载 Silero VAD 模型（只需一次）
model, utils = torch.hub.load(
    repo_or_dir='snakers4/silero-vad',
    model='silero_vad',
    force_reload=False
)
(get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils

# -------------------------------
# 函数：使用 VAD 检测语音段，返回总说话时长（秒）
# -------------------------------
def get_speech_duration(audio_path, threshold=0.5):
    try:
        # 读取并重采样到 16kHz
        wav, sr = librosa.load(audio_path, sr=16000, mono=True)
        wav = torch.from_numpy(wav).float()

        # 旧版参数：不能用 max_silence_duration_ms
        speech_timestamps = get_speech_timestamps(
            wav,
            model,
            threshold=threshold,
            min_speech_duration_ms=100,
            # max_silence_duration_ms=300,  # ❌ 删除这行
            window_size_samples=512  # 可选：设置帧大小（512, 1024, 1536）
        )

        total_duration = 0.0
        for seg in speech_timestamps:
            start_ms = seg['start']
            end_ms = seg['end']
            total_duration += (end_ms - start_ms) / 1000.0

        return total_duration

    except Exception as e:
        print(f"❌ Error processing {audio_path}: {str(e)}")
        return None

# -------------------------------
# 主流程：读取 CSV → 统计每条音频的说话时长
# -------------------------------
if __name__ == "__main__":
    if not os.path.exists(csv_file):
        raise FileNotFoundError(f"CSV file not found: {csv_file}")

    task = pd.read_csv(csv_file)

    if 'reference_speech' not in task.columns:
        raise ValueError("CSV 文件中缺少 'reference_speech' 列！")

    results = []

    for idx, row in task.iterrows():
        ref_name = row['reference_speech']

        # 处理扩展名
        base_name, ext = os.path.splitext(ref_name)
        if ext == "":
            audio_file = base_name + ".wav"
        else:
            audio_file = ref_name

        audio_path = os.path.join(audio_dir, audio_file)

        if not os.path.exists(audio_path):
            print(f"⚠️  File not found: {audio_path}")
            results.append({**row, 'speech_duration': None, 'error': 'file not found'})
            continue

        print(f"Processing: {audio_file}")
        # duration = get_speech_duration(audio_path, min_silence_duration=0.1, threshold=0.5)
        duration = get_speech_duration(audio_path, threshold=0.5)
        print(f"{float(duration):.3f}")

        # 保存结果
        results.append({**row, 'speech_duration(ms)': duration})

    # 转为 DataFrame 并保存
    result_df = pd.DataFrame(results)
    #[可配置]
    # output_csv = os.path.join(fold_root, "result_with_speech_duration.csv")
    output_csv = "./pre_deal/result_with_speech_duration.csv"
    result_df.to_csv(output_csv, index=False)

    print(f"\n✅ 语音时长统计完成！结果已保存至: {output_csv}")

    # 打印统计摘要
    valid_durations = [d for d in result_df['speech_duration(ms)'] if d is not None]

    if valid_durations:
        # 定义区间边界：0, 6, 12, ..., 60
        bins = np.arange(0, 66, 6)  # [0, 6, 12, ..., 60]
        labels = [f"{i}-{i+6}s" for i in range(0, 60, 6)]  # 标签：['0-6s', '6-12s', ...]

        # 统计每个区间的数量
        counts, _ = np.histogram(valid_durations, bins=bins)

        # 打印结果，数量转为 float 并保留 3 位小数（格式化为 3 位小数）
        print("\n" + "="*40)
        print("🗣️  人声说话时长分布 (0-60s, 每6秒一个区间)")
        print("="*40)
        for label, count in zip(labels, counts):
            print(f"{label:6s}: {float(count):.3f}")  # 整数转 float，保留 3 位小数

        # 附加摘要信息（可选）
        print("-" * 40)
        print(f"📊 总有效音频数: {len(valid_durations)}")
        print(f"📈 平均说话时长: {np.mean(valid_durations):.3f}s")
        print(f"🔍 最短: {min(valid_durations):.3f}s, 最长: {max(valid_durations):.3f}s")
        print("="*40)

Using cache found in /root/.cache/torch/hub/snakers4_silero-vad_master


Processing: reference_1.wav
48.486
Processing: reference_2.wav
49.952
Processing: reference_3.wav
87.520
Processing: reference_4.wav
65.920
Processing: reference_5.wav
67.488
Processing: reference_6.wav
50.464
Processing: reference_7.wav
59.168
Processing: reference_8.wav
21.856
Processing: reference_9.wav
82.880
Processing: reference_10.wav
87.360
Processing: reference_11.wav
90.048
Processing: reference_12.wav
83.616
Processing: reference_13.wav
390.624
Processing: reference_14.wav
52.192
Processing: reference_15.wav
50.120
Processing: reference_16.wav
63.072
Processing: reference_17.wav
346.912
Processing: reference_18.wav
131.488
Processing: reference_19.wav
65.184
Processing: reference_20.wav
32.632
Processing: reference_21.wav
44.079
Processing: reference_22.wav
45.600
Processing: reference_23.wav
32.544
Processing: reference_24.wav
102.848
Processing: reference_25.wav
115.552
Processing: reference_26.wav
353.888
Processing: reference_27.wav
115.488
Processing: reference_28.wav
1

In [8]:
import soundfile as sf

# 查看音频信息
info = sf.info("./AISumerCamp_audio_generation_fight/aigc_speech_generation_tasks/reference_102.wav")

print("文件信息:")
print(f"  采样率: {info.samplerate} Hz")
print(f"  通道数: {info.channels}")
print(f"  位深 (subtype): {info.subtype}")
print(f"  格式: {info.format}")
print(f"  时长: {info.duration:.2f} 秒")

文件信息:
  采样率: 16000 Hz
  通道数: 1
  位深 (subtype): PCM_16
  格式: WAV
  时长: 3.05 秒
